## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-10-10-52-38 +0000,bbc,Malian TikToker killed by suspected jihadists ...,https://www.bbc.com/news/articles/cnvep8ppyr6o...
1,2025-11-10-10-45-00 +0000,wsj,"U.S. Treasury Curve to Steepen Next Year, but ...",https://www.wsj.com/finance/investing/jgbs-edg...
2,2025-11-10-10-35-17 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/11/10/us/tru...
3,2025-11-10-10-35-14 +0000,bbc,Guinness maker appoints former Tesco boss to s...,https://www.bbc.com/news/articles/cy4021v7wepo...
4,2025-11-10-10-30-00 +0000,wsj,Italian Pasta Is Poised to Disappear From Amer...,https://www.wsj.com/world/europe/italy-pasta-t...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,31
199,shutdown,22
442,new,11
198,government,10
50,typhoon,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
175,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...,86
73,2025-11-10-05-02-42 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...,82
133,2025-11-09-22-28-52 +0000,nypost,Senate eyes vote on potential breakthrough to ...,https://nypost.com/2025/11/09/us-news/senate-e...,73
79,2025-11-10-04-14-40 +0000,nyt,Sean Duffy Juggles Shutdown Roles on Flight Sa...,https://www.nytimes.com/2025/11/09/us/politics...,65
82,2025-11-10-03-56-00 +0000,wsj,The Senate late Sunday cleared a critical proc...,https://www.wsj.com/politics/policy/republican...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
175,86,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...
82,43,2025-11-10-03-56-00 +0000,wsj,The Senate late Sunday cleared a critical proc...,https://www.wsj.com/politics/policy/republican...
138,40,2025-11-09-22-01-33 +0000,wapo,Super Typhoon Fung-wong slams Philippines in w...,https://www.washingtonpost.com/world/2025/11/0...
69,35,2025-11-10-05-22-38 +0000,wapo,Special counsel indicts former South Korean pr...,https://www.washingtonpost.com/world/2025/11/1...
116,33,2025-11-10-00-09-50 +0000,nypost,Brooklyn dad says goodbye to hero son Itay Che...,https://nypost.com/2025/11/09/world-news/nyc-d...
174,30,2025-11-09-18-48-33 +0000,cbc,"BBC director, head of news resign after critic...",https://www.cbc.ca/news/world/bbc-director-res...
46,29,2025-11-10-07-48-33 +0000,nypost,Madman steals FedEx truck on NYC street and go...,https://nypost.com/2025/11/10/us-news/maniac-s...
27,24,2025-11-10-09-46-00 +0000,wsj,Rumble to Acquire Northern Data in Deal Valued...,https://www.wsj.com/business/deals/rumble-to-a...
13,23,2025-11-10-10-03-27 +0000,nyt,Prison Guards Shaved His Dreadlocks. The Supre...,https://www.nytimes.com/2025/11/10/us/supreme-...
134,23,2025-11-09-22-18-20 +0000,bbc,Harshita was murdered a year ago - so why has ...,https://www.bbc.com/news/articles/c993je80ermo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
